In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
Y_train = train.label
X_train = train.drop(columns=['label'], axis=1)

In [ ]:
import seaborn as sns
sns.countplot(Y_train)

### Normalization

In [ ]:
X_train /= 255.0
test /= 255.0

### Reshape

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

### label encoding

In [ ]:
from keras.utils.np_utils import to_categorical
Y_train = to_categorical(Y_train, num_classes=10)

### split train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)

In [ ]:
print(X_train[0].shape)
plt.imshow(X_train[0][:, :, 0])
plt.show()

### CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learn_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
epochs = 30
batch_size = 86

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val), verbose=2)

### Evaluate the model

In [ ]:
Y_pred = model.predict(X_val)

In [ ]:
Y_pred_classes = np.argmax(Y_pred, axis=1)

In [ ]:
Y_true = np.argmax(Y_val, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(Y_true, Y_pred_classes)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(Y_true, Y_pred_classes)

In [ ]:
results = model.predict(test)

In [ ]:
results = np.argmax(results, axis=1)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')

In [ ]:
submission.Label = results

In [ ]:
submission.to_csv('output.csv', index=False)